In [1]:
### 问题一
## 1.常规思路

import numpy as np
from pyqubo import Array, Constraint, Placeholder
import pandas as pd
from openjij import SQASampler

# 贷款利息收益率
R = 0.08

# 约束系数 M
M = 2

# matrix shape
N1 = 10
N2 = 100

# 读取附件 csv
ht = pd.read_csv('data_100.csv') 
hv = ht.values
h1 = []
t1 = []

# 建立 t, h Array
for o in range(hv.shape[1]):
    if (o%2 == 0):
        t1.append(hv[:,o])
    else:
        h1.append(hv[:,o])

t = np.array([np.empty(N1)]*N2)
h = np.array([np.empty(N1)]*N2)

for k in range(N2):
    for l in range(N1):
        t[k][l] = t1[k][l]
        h[k][l] = h1[k][l]

# 建立 x array
x = Array.create('x', (N2, N1), 'BINARY')

## 定义约束条件和目标函数
# 约束条件 1:
# 在 100 张卡中 选且仅能选 1 张信用评分卡（共计出现一个 1）
H1 = np.sum([(np.sum([x[i, j] for i in range(N2) for j in range(N1)]) - 1)**2])

# 计算 H0(Loss)
H0 = np.sum([(1.08 * h[i, j] * t[i, j] - R * t[i, j]) * x[i, j]\
            for i in range(N2) for j in range(N1)])

# 目标函数
H = H0 + Placeholder('M') * Constraint(H1, label='H1')
model = H.compile()

feed_dict = {'M': M}

# 求出 QUBO 矩阵
qubo, offset = model.to_qubo(feed_dict=feed_dict)


# Sample(使用 SQASampler 模拟退火)
sampler = SQASampler()

# num_reads
R = 2000

sampleset = sampler.sample_qubo(qubo, num_reads=R)

# Save to ./xx/xxx_x_xxx.csv
save_path = './save_csv/ans_1_1_{}iter_2.csv'.format(R)
save = pd.DataFrame(sampleset)
save.to_csv(save_path)

# 输出结果
index_card = []

for i, row in save.iloc[[0],:].iterrows():
    for j in range(len(row)):
        if row[j] == 1:
            outrow = row.index[j]
            index_card.append(row.index[j].split('x[')[1])

ans_1 = index_card[0].split('][')[0]
ans_2 = index_card[0].split('][')[1].split(']')[0]
print('选择第 {} 张信用评分卡的第 {} 个阈值时最终收入最多'.format(int(ans_1) + 1, int(ans_2) + 1))

选择第 49 张信用评分卡的第 2 个阈值时最终收入最多


C:\Users\Administrator\AppData\Local\Temp\ipykernel_33248\3494587703.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[j] == 1:
